<a href="https://colab.research.google.com/github/JSaillok/ArxesGlwsswn/blob/main/porject_info_retrieval_1064886.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set elasticsearch and tensorflow

Εισαγωγή των απαραίτητων βιβλιοθηκών για το project και αρχικοποίηση του elasticsearch.

In [ ]:
!pip install tensorflow-io
!pip3 install elasticsearch==6.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 1.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112 kB 10.3 MB/s 
     |████████████████████████████████| 132 kB 38.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import os
import time
from sklearn.model_selection import train_test_split
from elasticsearch import Elasticsearch, helpers
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow_io as tfio

/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIFN3tsl8StatusOrISt10unique_

In [ ]:
print("tensorflow-io version: {}".format(tfio.__version__))
print("tensorflow version: {}".format(tf.__version__))

tensorflow-io version: 0.28.0
tensorflow version: 2.9.2


In [ ]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
# Sleep for few seconds to let the instance start.
time.sleep(20)

In [ ]:
%%bash

ps -ef | grep elasticsearch

root         335     333  0 00:00 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       336     335 99 00:00 ?        00:00:23 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-15119995131702870432 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:fileco

In [ ]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "467b97bfceac",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "v2qTHZn2TROlLUUNCZPJyA",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


# Task 1


Merge τα 3 csv αρχεία

In [ ]:
import os
import glob
import pandas as pd
os.chdir("/content/drive/MyDrive")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
merged = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
merged.to_csv( "merged.csv", index=False, encoding='utf8')

<ipython-input-9-e27c01416e3c>:10: DtypeWarning: Columns (1,3,5,6,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  merged = pd.concat([pd.read_csv(f) for f in all_filenames ])



Upl.csv αρχείου στο elasticsearch.

In [ ]:
import csv

ES_NODES = "http://localhost:9200"

es = Elasticsearch(hosts = [ES_NODES])

with open("/content/drive/MyDrive/BX-Books.csv", encoding="utf8") as f:
	reader = csv.DictReader(f)
	helpers.bulk(es, reader, index='bx-books')

with open("/content/drive/MyDrive/BX-Users.csv", encoding="utf8") as f1:
	reader = csv.DictReader(f1)
	helpers.bulk(es, reader, index='bx-users')

with open("/content/drive/MyDrive/BX-Book-Ratings.csv", encoding="utf8") as f2:
	reader = csv.DictReader(f2)
	helpers.bulk(es, reader, index='bx-book-ratings')

print("Upload Completed")

Upload Completed


In [ ]:
%%bash

curl -sX GET "localhost:9200/_cat/indices"

yellow open bx-books        JaAPWczmQ5OlMFbpNJnPKQ 1 1  18765 0   16mb   16mb
yellow open bx-book-ratings CB47iXqBQsaUZyrip6RhUg 1 1 248500 0 11.2mb 11.2mb
yellow open bx-users        IRpbtmQ-Tjyta32RbN-u4A 1 1 130373 0 11.5mb 11.5mb


In [ ]:
%%bash

curl -sX GET "localhost:9200/bx-users/_search" -sH 'Content-Type: application/json' -sd'{  "size": 2,  "query": {    "match_all": {}  }}'

{"took":101,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":10000,"relation":"gte"},"max_score":1.0,"hits":[{"_index":"bx-users","_type":"_doc","_id":"biBW-YQBx5R9hzMic2_T","_score":1.0,"_source":{"uid": "1", "location": "nyc, new york, usa", "age": ""}},{"_index":"bx-users","_type":"_doc","_id":"byBW-YQBx5R9hzMic2_T","_score":1.0,"_source":{"uid": "2", "location": "stockton, california, usa", "age": "18.0"}}]}}

Αναζήτηση βάση αλφαριθμητικό string

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import pandas as pd

ES_NODES = "http://localhost:9200"

es = Elasticsearch(hosts = [ES_NODES])

#alphanumeric + integer
userId = input("Give the user ID: \n")

#alpanumeric input
keyword = input("Give a string: \n")

def getBook():

  # Elasticsearch Query.
	query = {
		"query": {
			"match": {
				"book_title": {
					"query": keyword,
					"operator": "and"
				}
			}
		}#,
    #"aggs": {
      #"book_count" : {
          #"value_count" : { 
              #"field" : "type" 
              #} 
            #}
          #}
  }

  # Collecting all the data using Scan function.
	rel = scan(
		client=es,
		query=query,
		scroll='1m',
		index='bx-books',
		raise_on_error=True,
		preserve_order=False,
		clear_scroll=True
	)
 
 	# This list keeps the response.
	result = list(rel)
	temp = []

  # Clear the response data from metadata like _id, _type, _index.
	for hit in result:
		# So, we're getting only '_source'
		temp.append(hit['_source'])

	# Create a dataframe
	books_df = pd.DataFrame(temp)

	return books_df



def getRatings():
  # Elasticsearch Query.
	query = {
		"query": {
			"match": {
				"uid": {
					"query": userId
				}
			}
		}
	}

  # Collecting all the data using Scan function.
	rel1 = scan(
		client=es,
		query=query,
		scroll='1m',
		index='bx-book-ratings',
		raise_on_error=True,
		preserve_order=False,
		clear_scroll=True
	)
 
 	# This list keeps the response.
	result = list(rel1)
	temp1 = []

  # Clear the response data from metadata like _id, _type, _index.
	for hit in result:
		# So, we're getting only '_source'
		temp1.append(hit['_source'])

	# Create a dataframe
	user_df = pd.DataFrame(temp1)

	return user_df

# Calling the function
df = getBook()
df1 = getRatings()

# Sorting the Dataframe
sorted_df = df.sort_values(["book_title"], ascending=False)


print(sorted_df)
print('\n\n')
print(df1)
print('\n')

mergedPd = pd.merge(sorted_df, df1)
print(mergedPd)
print('\n\n')

Give the user ID: 
8
Give a string: 
Canada
         isbn                                         book_title  \
1  0771095066  Poets of Contemporary Canada (New Canadian Lib...   
2  0968067824                                   Crazy For Canada   
3  0919891276  Book of Alternative Services of the Anglican C...   
0  1553373405                                      ABC of Canada   

            book_author year_of_publication                 publisher  \
1            Eli Mandel                1972      New Canadian Library   
2          Noa Schwartz                1997          Tumbleweed Press   
3  Not Applicable (Na )                1988  Continuum Intl Pub Group   
0     Kim Bellefontaine                2002            Kids Can Press   

                                             summary                category  
1  This important anthology affirms the continuat...              ['poetry']  
2  Lester B. Beaver describes his vacation in Can...    ['juvenile fiction']  
3  The pew e